![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidget_6 Buttons and LEDs

This short notebook introduces how we access a Phidgets device from a Jupyter notebook.

We will monitor the two pushbuttons on the Phidget starter kit, and use them to control the LEDs on that box. 

It is important to run this notebook in the Google Chrome web browser, on a desktop or laptop computer. Opera and Microsoft Edge should work as well. Unfortunately, other browsers (FireFox, Safari) do not work currently, as they cannot access the USB port that connects to the Phidget device.


Reference code: https://www.phidgets.com/education/learn/getting-started-kit-tutorial/blink-led/
Be sure to look at the JavaScript code.

## Setting up the Phidget device

We will be using the Phidget Starter Kit, as shown in this figure:
<img src="images/KIT4003.jpg" alt="Image of the starter kit"  width = 500 />

Verify these points:
- Get a hold of a Phidgets Starter Kit. It should already be assembled. If not, assemble it now.
    - You will have received instructions with the kit on how to assemble it.
- The Starter Kit has a two buttons and two switches connect. If it does not, find them and attach.
    - We assume the red button is attached to port 0, and the red LED attached to port 1.
    - We assume the green button is attached to port 5, and the green LED attached to port 4.
    - if note, either change yoor wires or chage the "setPortHub" values in the code below. 
- Connect the Starter Kit to your computer with a USB cable. 
    - The Kit come with a USB cable, it is attached to the Hub device inside the Starter Kit.
- If you have an older version of the Phidgets, you may need to update its firmware. See the following for details:
    - https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
- As mentioned above, make sure you are running this notebook in the Google Chrome browser. 

## Setup the Phidget device

As in the earlier notebooks, the main steps are thus:

1. Load in some Python libraries
2. Create a widget using **anywidget** object, with details in Javascript 
    - Load in some external JS libraries
    - Define and open the USB connection
    - Open the digital inputs (buttons) and outputs (LEDS). There are four in total
    - Request a connection to the Phidgets device. Include code to close the devices as well.
    - Define the user interface with buttons and text. 
    
3. Display the widget
4. Make the connection to the USB devices.
5. Read values from the buttons, push values to the LEDs 
10. When we are done, close the device

## Step 1.

Let's import a few Python libraries that we need. 

The **anywidget** library is still in development, so we do a pip install if it is not already on the hub.  

In [1]:
%pip install anywidget

In [2]:
import anywidget

## NOTE:

If the **install** or **import** above gives an error message, try running them aghain. If that doesn't work, try closing the notebook, shut down the kernel, then reopen the notebook. Run the cell again. After that, the library will be properly installed.  

## Step 2.

We create the PhidgetButtons class (software object) as an example of the **anywidget** models. The JS code for this tool is shown. Have a read, and feel free to experiment with it. 


In [3]:
class PhidgetButtons(anywidget.AnyWidget):
  _esm = """

// Importing the functions we need for Phidgets
  import {USBConnection, DigitalInput, DigitalOutput} from "https://esm.sh/phidget22@3.17";

  export function render({ model, el }) {
    
    var connection = 0, inA = 0, inB = 0, outA = 0, outB = 0, connOpen = false;

// Opening the USBconnection
    async function openUSB() {
        if (connection) {
            try {await connection.close();} catch {}
            try {await connection.delete();} catch {}
        }
        try {connection = new USBConnection();} 
            catch{console.log("This browser does not support WebUSB");}

     	try {
                await connection.connect();
                connection.requestWebUSBDeviceAccess();
                connOpen = true;
        } catch(err) {
                connOpen = false;
        }

// Opening the digital inputs (for switched) and outputs (for LEDs)
        inA = new DigitalInput();
        inA.setHubPort(0);        
        inA.setIsHubPortDevice(true);
        inA.onStateChange = function(state) {
            if (state) {textA.innerHTML = 'Button A is PUSHED.  .';}
            else {textA.innerHTML = 'Button A is not pushed.  .';}
         }

        inB = new DigitalInput();
        inB.setHubPort(5);        
        inB.setIsHubPortDevice(true); 
        inB.onStateChange = function(state) {
            if (state) {textB.innerHTML = 'Button B is PUSHED.';}
            else {textB.innerHTML = 'Button B is not pushed.';}
         }

        outA = new DigitalOutput();
        outA.setHubPort(1);        
        outA.setIsHubPortDevice(true); 

        outB = new DigitalOutput();
        outB.setHubPort(4);        
        outB.setIsHubPortDevice(true); 

// Here is the code to connect the devices
     	try {
                await inA.open(1000);
                await inB.open(1000);
                await outA.open(1000);
                await outB.open(1000);
                connOpen = true;
        } catch(err) {
                connOpen = false;
        }
       
         // now get the UI to reflect the status
        if (connOpen) {
            textStatus.innerHTML = 'USB connected, channels open.  .';
            button.innerHTML = `Click to disconnect`;
        }
        else {
            textStatus.innerHTML = 'USB did not connect.  .';
            button.innerHTML = `Click to connect`;        
        }
    };

    async function closeUSB() {
        try {await connection.close();} catch {}
        try {await connection.delete();} catch {}
        connOpen = false;
        textStatus.innerHTML = 'Disconnected.  .';
        button.innerHTML = `Click to connect`;
    };
    
// The user interface, with buttons and text 
    let button = document.createElement("button");
    button.classList.add("ph-button");
    button.innerHTML = `Click to open USB`;
    button.addEventListener("click", async () => {
        if (connOpen) {closeUSB();} else {openUSB();}
    });
    let buttonA = document.createElement("button");
    buttonA.classList.add("ph-button");
    buttonA.innerHTML = `Click for LED A`;
    buttonA.addEventListener("mousedown", () => {
        outA.setState(true);
    });
    buttonA.addEventListener("mouseup", () => {
        outA.setState(false);
    });
    let buttonB = document.createElement("button");
    buttonB.classList.add("ph-button");
    buttonB.innerHTML = `Click for LED B`;
    buttonB.addEventListener("mousedown", () => {
        outB.setState(true);
    });
    buttonB.addEventListener("mouseup", () => {
        outB.setState(false);
    });
    let textStatus = document.createElement("label");
    textStatus.innerHTML = 'Status message here.  .';
    let textA = document.createElement("label");
    textA.innerHTML = 'Button A is not pushed.  .';
    let textB = document.createElement("label");
    textB.innerHTML = 'Button B is not pushed.';
    
// Now display the user interface
 
    el.appendChild(button);
    el.appendChild(buttonA);
    el.appendChild(buttonB);
    el.appendChild(textStatus);
    el.appendChild(textA);
    el.appendChild(textB);
    
// Clean up when the notebook is closed
    return async () => {
        try {await channel.close();} catch {}
        try {await connection.close();} catch {}
        try {await connection.delete();} catch {}
    };
  }
  """
  _css = """
        .ph-button {color: white; 
            background-color:rgb(96, 107, 174); 
            border-radius: 8px; 
            font-size: 24px; 
            display: block;
            padding: 15px 32px;}
        .ph-button:hover { background-color:rgb(120, 128, 187); }
  """

pb = PhidgetButtons()


## Step 3.

We now open and display the Phidget widget with all its buttons and text. Run the following cell.

In [4]:
display(pb)

PhidgetButtons()

## Step 4. 
Clicking on the "Connect" button above will open a window that asks you to select the Phidget Hub. Select the hub and click "Connect." The window looks like this:
<img src="images/Pconnect.png" alt="Hey"  width = 300 />

If you don't get this window, perhaps you are using the wrong browser. (Use Chrome, Chromium, or Opera).

If you get the window but don't see the Phidget device, check to be sure the Phidget VINT hub is connected to your computer with a USB cable.

### Check:
If all is well, the status text will report that the USB port is connected and the device channels are open. You should also see buttons for the LED and text boxes for the state of the push buttons on the Phidget hardware. 



If you don't see any values, check again that the Phidget VINT device is connected to your computer with a USB cable. Also check that you have the LEDs and pushbottom cables connected to the correct ports, as per the setup instruction. 

- We assume the red button (inA) is attached to port 0, and the red LED (outA) attached to port 1.
- We assume the green button (inB) is attached to port 5, and the green LED (outB) attached to port 4.

If these are different on your device, you will need to change these numbers in the JS code above to match the ports on the VINT USB hub. 


## Step 5.

Now try it out.

Click on the buttons on the screen to light the LEDs.

Push the pushbuttons on the Phidget hardware and observe the text on the screen reporting that the buttons are pushed. 

## Step 6. Closing down the sensors

It is **really important** to close the Phidget device when you are done. Otherwise they will keep busy forever, always trying to update the text with the latest values. Also, no other notebook can use the Phidgets while this one is active. 

To close the Phidget, go back to the Connect button above, and click where it says "Click to disconnect."

You can also simply close this notebook. 

## In case of errors

Do you see buttons appear on the screen, but nothing is happening? Go back and look for the "Connect" button just above Step 4. Click on it to get the Phidget hardware to connect to the software.

Still having problems? What kind of errors can happen? What should we do about them?

**Symptom 1**. When you click the Connect button, status text says "USB not connected"
- Problem is likely that your browser does not support WebUSB.
    - Solution is to switch to the latest versions of Chrome, Chromium, Opera or Edge.
- Another problem could be that the Phidget VINT device is not connected via a USB cable. You should see a window like this <img src="images/Pconnect.png" alt="Hey"  width = 300 /> Select the VINT device once it appears.
- Another notebook or piece of software is already connected to the Phidget device.
    - Only one notebook at a time can use the Phidget. So close down any other notebook using Phidgets, or exit any other software using the device. 
- You might have an older version of a Phidget that needs a firmware update.
    - see here for the solution: https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
  
    
**Symptom 2**. When you click the Connect button, status text is "USB connected, channel not open."
- Check to see that you have a Phidget temperature/humidity device connected to your Phidget hub or VINT device


## And that's it. 

The Phidget device will continue to monitor the two push buttons, display the result on the screen, and light up the appropriate LEDs. .


Good job!

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)